In [3]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [4]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = '../docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/var/folders/q8/wt86p9kd3rd1ksjbgp7kvqt80000gn/T/ipykernel_12212/1475080136.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
/var/folders/q8/wt86p9kd3rd1ksjbgp7kvqt80000gn/T/ipykernel_12212/1475080136.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [6]:
print(vectordb._collection.count())

208


In [7]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [8]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

/var/folders/q8/wt86p9kd3rd1ksjbgp7kvqt80000gn/T/ipykernel_12212/60327743.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=llm_name, temperature=0)


In [9]:
from langchain.chains import RetrievalQA

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [11]:
result = qa_chain({"query": question})

/var/folders/q8/wt86p9kd3rd1ksjbgp7kvqt80000gn/T/ipykernel_12212/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [12]:
result["result"]

'The major topics for this class include machine learning, statistics, and algebra. Additionally, there will be discussions on extensions of the material covered in the main lectures.'

In [13]:
# Prompt

In [14]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [15]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [16]:
question = "Is probability a class topic?"

In [17]:
result = qa_chain({"query": question})

In [18]:
result["result"]

'Yes, probability is a topic assumed to be familiar to students in the class, as mentioned by the instructor. Thanks for asking!'

In [19]:
result["source_documents"][0]

Document(metadata={'creator': 'PScript5.dll Version 5.2.2', 'source': '../docs/MachineLearning-Lecture01.pdf', 'creationdate': '2008-07-11T11:25:23-07:00', 'moddate': '2008-07-11T11:25:23-07:00', 'author': '', 'page_label': '5', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'total_pages': 22, 'title': '', 'page': 4}, page_content="of this class will not be very programming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octave. I'll say a bit more about that later.  \nI also assume familiarity with basic probability and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what random variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites

In [20]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [21]:
result = qa_chain_mr({"query": question})

In [22]:
result["result"]

'Yes, probability is a class topic in the context of machine learning algorithms. The instructor mentions using a probabilistic interpretation to derive the next learning algorithm, indicating that probability is covered in the class.'

In [23]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'The original answer still adequately addresses the question about probability being a class topic in the context of machine learning. The instructor, Andrew Ng, uses a probabilistic interpretation to derive learning algorithms, which will be discussed in the main lectures. Additionally, the discussion sections will cover related topics such as statistics and algebra as refreshers, and extensions to the main lecture material will also be explored.'

In [24]:
"""
RetrievalQA limitations
QA fails to preserve conversational history.
"""

'\nRetrievalQA limitations\nQA fails to preserve conversational history.\n'

In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [26]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic in the course being described. The instructor assumes familiarity with basic probability and statistics, so it is likely that probability concepts will be covered in the class.'

In [28]:
"""
Maybe the model is updated, it returns correct answer even without the conversational history but below question related to TA
are better example for the RetrievalQA limitations
"""
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites mentioned in the context are needed because the course assumes familiarity with basic concepts in probability and statistics, as well as basic linear algebra. Understanding these concepts is essential for grasping the machine learning algorithms and techniques that will be covered in the course.'

In [29]:
question = "Who are the teaching assistants of the course?"
result = qa_chain({"query": question})
result["result"]

'The teaching assistants mentioned in the context are Paul Baumstarck, Catie Chang, Tom Do, Zico Kolter, and Daniel Ramage.'

In [33]:
"""
Please note this return about the 'qualifications of the individuals in the class' rather than TAs as does not have conversational 
memory hence this is the limitation that we meant.
"""
question = "What is there qualification?"
result = qa_chain({"query": question})
result["result"]

'Based on the context provided, the qualifications of the individuals in the class include majors in computer science, electrical engineering (EE), iCME (Institute for Computational and Mathematical Engineering), MSNE (Materials Science and Engineering), and other fields like aerospace/astro, chemistry, and industry. The class seems to have a diverse range of backgrounds and qualifications.'